# Estrazione delle quadruple (aspect-opinion-category-sentiment) con ModernBERT su Restaurant-ACOS

In [ ]:
# Import delle librerie necessarie
import torch
import numpy as np
import random
import pandas as pd
import wandb
import os
import sys
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, get_linear_schedule_with_warmup, AutoModel
from torch.optim import AdamW
import torch.nn as nn
import bitsandbytes as bnb
import pickle
from evaluate import load
from tqdm import tqdm
from torch.amp import autocast, GradScaler # Per Mixed Precision
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score

# --- 1. CONFIGURAZIONE DEL DEVICE ---
# Se hai una GPU NVIDIA, userà 'cuda'. Se hai un Mac M1/M2, userà 'mps'. Altrimenti 'cpu'.
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f" GPU Trovata: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(" Acceleratore Apple Metal (MPS) Trovato")
else:
    device = torch.device("cpu")
    print(" Nessuna GPU trovata. L'addestramento sarà lento.")

print("Librerie caricate.")

 Acceleratore Apple Metal (MPS) Trovato
Librerie caricate.


### Impostazioni per la riproducibilità 

In [2]:
def set_seed(seed_value=42):
    """Imposto i seed per la riproducibilità."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        # Imposto anche i seed per la GPU, se disponibile
        torch.cuda.manual_seed_all(seed_value)

# Esegui l'impostazione del seed
set_seed(42) 
print("Random seeds impostati su 42.")

Random seeds impostati su 42.


In [ ]:
if wandb.run is not None:
    wandb.finish()

WANDB_ENTITY = "cristinatextmining"

# 1. Definizione degli Hyperparameters
config = {
    "learning_rate": 2e-5,
    "epochs": 40,
    "batch_size": 16,
    "model_name": "answerdotai/ModernBERT-base",
    "dataset": "Restaurant-ACOS",  
    "seed": 42,
    'patience': 5  # Per Early Stopping
}

# 2. Inizializzazione del Run
wandb.init(
    project="BigData-TextMining-ACOS",
    entity=WANDB_ENTITY,
    config=config,
    name=f"Step1_Class_{config['model_name']}_{config['dataset']}" 
)

print(f"W&B inizializzato per il progetto: {wandb.run.project}")
print(f"Nome della Run attuale: {wandb.run.name}")

wandb: Currently logged in as: cristinatomaciello2001 (cristinatextmining) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B inizializzato per il progetto: BigData-TextMining-ACOS
Nome della Run attuale: run_answerdotai/ModernBERT-base_Restaurant-ACOS


## PyTorch Dataset & DataLoader Construction

### Creazione di PyTorch Dataset e DataLoader
Questa cella si occupa di caricare i dati pre-processati e di "impacchettarli" nel formato esatto richiesto dalla nostra nuova architettura PyTorch personalizzata. Rappresenta un passaggio cruciale per replicare fedelmente il paper originale, permettendoci di gestire gli elementi impliciti.

Nello specifico, il codice esegue tre operazioni fondamentali:

1. **Caricamento dei DataFrame:** Legge i file `.pkl` (Train, Dev e Test per il dominio Laptop) che abbiamo precedentemente aggiornato. Questi file ora contengono le annotazioni binarie che indicano la presenza di aspetti o opinioni implicite.
2. **Definizione della Classe Custom `ACOSDataset`:** Questa è la modifica principale rispetto a una pipeline standard di HuggingFace. Invece di restituire solo i classici 3 tensori della Token Classification (`input_ids`, `attention_mask` e `labels` con i tag BIO), questa classe sovrascritta restituisce **5 tensori** per ogni frase. Vengono infatti estratti e passati al modello anche `implicit_aspect_labels` e `implicit_opinion_labels`. Questi tensori serviranno ad addestrare in parallelo le due teste di classificazione binaria sul token `[CLS]`.
3. **Configurazione dei DataLoader:** Crea gli iteratori di PyTorch che alimenteranno il modello durante l'addestramento e il test, processando blocchi (batch) di 16 frasi alla volta. I dati di training vengono rimescolati (`shuffle=True`) per stabilizzare l'apprendimento della rete.

In [3]:
# --- 1. CARICAMENTO DEI DATASET SALVATI  ---
cartella_dati = "data_allineati"

print("Caricamento dei dataset pre-processati...")
# Carichiamo i Ristoranti
df_train_align_rest = pd.read_pickle(os.path.join(cartella_dati, "train_rest_aligned.pkl"))
df_dev_align_rest = pd.read_pickle(os.path.join(cartella_dati, "dev_rest_aligned.pkl"))
df_test_align_rest = pd.read_pickle(os.path.join(cartella_dati, "test_rest_aligned.pkl"))


class ACOSDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['labels'].tolist()
        # Estraiamo le colonne per gli impliciti!
        self.implicit_aspect_label = df['implicit_aspect_label'].tolist()
        self.implicit_opinion_label = df['implicit_opinion_label'].tolist()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
            # Passiamo le etichette al Dataloader
            'implicit_aspect_labels': torch.tensor(self.implicit_aspect_label[idx], dtype=torch.long),
            'implicit_opinion_labels': torch.tensor(self.implicit_opinion_label[idx], dtype=torch.long)
        }

# --- CREAZIONE DELLE ISTANZE ---

# Creiamo i dataset per il dominio restaruant
train_dataset_rest = ACOSDataset(df_train_align_rest)
dev_dataset_rest = ACOSDataset(df_dev_align_rest)
test_dataset_rest = ACOSDataset(df_test_align_rest)

# --- CONFIGURAZIONE DATALOADERS ---

BATCH_SIZE = 16 # Numero di frasi analizzate contemporaneamente

train_loader_rest = DataLoader(train_dataset_rest, batch_size=BATCH_SIZE, shuffle=True)
dev_loader_rest = DataLoader(dev_dataset_rest, batch_size=BATCH_SIZE)
test_loader_rest = DataLoader(test_dataset_rest, batch_size=BATCH_SIZE)

print(f"Dataset e DataLoaders creati con successo!")
print(f"Esempi nel set di Training RESTAURANT: {len(train_dataset_rest)}")  

Caricamento dei dataset pre-processati...
Dataset e DataLoaders creati con successo!
Esempi nel set di Training RESTAURANT: 1530


### Architettura Multi-Task: ModernBERT ACOS Extractor

In questa cella abbandoniamo l'architettura standard di Token Classification per costruire un modello personalizzato in PyTorch, progettato per replicare fedelmente la logica di estrazione del paper ACOS originale.

Il problema dei modelli tradizionali è che falliscono quando un aspetto o un'opinione non sono scritti esplicitamente nel testo (elementi *Impliciti*). Per risolvere questa criticità, abbiamo progettato una rete **Multi-Task** composta da un "cervello" centrale (l'encoder ModernBERT) e **tre teste di classificazione indipendenti**:

1. **Testa di Token Classification (Estrazione Esplicita):** Analizza ogni singola parola della frase per assegnare i tag BIO (B-ASP, I-ASP, B-OPI, I-OPI, O), estraendo gli span di testo espliciti.
2. **Testa per Aspetti Impliciti (Classificazione Binaria):** Sfrutta il token speciale `[CLS]`, che racchiude il significato globale della frase, per prevedere matematicamente (Sì/No) se la recensione contiene un aspetto sottinteso.
3. **Testa per Opinioni Implicite (Classificazione Binaria):** Sfrutta sempre il token `[CLS]` per indovinare se c'è un'opinione sottintesa.

**La Loss Combinata (L'addestramento simultaneo)**
Il vero "motore" di questa classe è nella funzione `forward`. Durante l'addestramento, il modello calcola contemporaneamente tre errori separati (uno per l'estrazione e due per le previsioni binarie degli impliciti). Questi tre errori vengono **sommati in un'unica Loss globale**. In questo modo, la rete neurale viene forzata a imparare tutti i task simultaneamente, ottimizzando i pesi interni per comprendere a fondo sia ciò che è scritto, sia ciò che è sottinteso.

Infine, il modello viene caricato sulla GPU e accoppiato a un ottimizzatore **AdamW a 8-bit** (`bitsandbytes`) per massimizzare l'efficienza e prevenire l'esaurimento della memoria VRAM durante le epoche.

In [ ]:
# Definiamo le 5 etichette: 0=O, 1=B-ASP, 2=I-ASP, 3=B-OPI, 4=I-OPI
NUM_LABELS = 5 
class ModernBertACOS_Extractor(nn.Module):
    def __init__(self, model_name="answerdotai/ModernBERT-base", num_labels=5):
        super().__init__()
        # Carichiamo la "schiena" del modello (l'encoder base)
        self.bert = AutoModel.from_pretrained(model_name)
        hidden_size = self.bert.config.hidden_size
        
        # Testolina 1: Trova le parole scritte (Token Classification)
        self.token_classifier = nn.Linear(hidden_size, num_labels)
        
        # Testoline 2 e 3: Indovinano se ci sono impliciti (Classificazione Binaria)
        self.implicit_aspect_classifier = nn.Linear(hidden_size, 2)
        self.implicit_opinion_classifier = nn.Linear(hidden_size, 2)
        
    def forward(self, input_ids, attention_mask, labels=None, implicit_aspect_labels=None, implicit_opinion_labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state 
        
        # Prendiamo il token [CLS] (posizione 0) per i classificatori binari
        cls_output = sequence_output[:, 0, :] 
        
        # Le tre testoline fanno le loro previsioni
        token_logits = self.token_classifier(sequence_output)
        imp_asp_logits = self.implicit_aspect_classifier(cls_output)
        imp_opi_logits = self.implicit_opinion_classifier(cls_output)
        
        loss = None
        # Calcolo della "Loss Combinata" durante il training
        if labels is not None and implicit_aspect_labels is not None and implicit_opinion_labels is not None:
            # Estraiamo il device (CUDA o CPU) per creare i tensori dei pesi nel posto giusto
            device = input_ids.device
            
            # --- NOVITÀ 1: Pesi per i Token (Lotta allo sbilanciamento delle 'O') ---
            # Indice 0 ('O') ha peso 1.0. 
            # Gli indici 1, 2, 3, 4 (Aspetti e Opinioni) hanno peso 10.0! Sbagliarli costa carissimo.
            token_weights = torch.tensor([1.0, 10.0, 10.0, 10.0, 10.0], device=device)
            loss_fct_token = nn.CrossEntropyLoss(weight=token_weights)
            
            # --- NOVITÀ 2: Pesi per gli Impliciti ---
            # Classe 0 (Esplicito) ha peso 1.0. Classe 1 (Implicito) ha peso 5.0.
            # Diciamo al modello: "Non ignorare gli impliciti solo perché sono rari!"
            implicit_weights = torch.tensor([1.0, 5.0], device=device)
            loss_fct_implicit = nn.CrossEntropyLoss(weight=implicit_weights)
            
            # Loss 1: Token (ignorando il padding -100)
            active_loss = attention_mask.view(-1) == 1
            active_logits = token_logits.view(-1, 5)[active_loss]
            active_labels = labels.view(-1)[active_loss]
            loss_token = loss_fct_token(active_logits, active_labels)
            
            # Loss 2 & 3: Impliciti
            loss_asp = loss_fct_implicit(imp_asp_logits, implicit_aspect_labels)
            loss_opi = loss_fct_implicit(imp_opi_logits, implicit_opinion_labels)
            
            # --- NOVITÀ 3: Moltiplicatori della Loss Multi-Task ---
            # Visto che il Recall delle opinioni implicite era solo il 45%, 
            # diamo peso doppio alla sua Loss nella somma totale!
            loss = loss_token + (1.5 * loss_asp) + (2.0 * loss_opi)
            
        return {
            "loss": loss, 
            "token_logits": token_logits, 
            "imp_asp_logits": imp_asp_logits, 
            "imp_opi_logits": imp_opi_logits
        }

print("Scaricamento e configurazione della nuova architettura Multi-Task ModernBERT...")

# Inizializziamo il nostro modello custom invece di AutoModelForTokenClassification
model_step1 = ModernBertACOS_Extractor(num_labels=NUM_LABELS)

# Spostiamo il modello sul dispositivo di calcolo (GPU/MPS/CPU)
model_step1.to(device)

# --- 2. CONFIGURAZIONE DELL'OTTIMIZZATORE E DELLA LOSS ---

# Manteniamo la tua ottima scelta di usare l'optimizer a 8-bit per non saturare la memoria!
optimizer = bnb.optim.AdamW8bit(model_step1.parameters(), lr=2e-5)



print("\n" + "="*50)
print("MODELLO MULTI-TASK PRONTO PER IL TRAINING")
print("="*50)
print(f"Architettura: ModernBERT-base (Custom ACOS Extractor)")
print(f"Task: Token Classification + 2x Binary Classification (Impliciti)")
print(f"Numero di Classi Token: {NUM_LABELS}")
print(f"Optimizer: AdamW 8-bit (lr=2e-5)")
print(f"Loss Function: Loss Combinata (calcolata internamente)")


Scaricamento e configurazione della nuova architettura Multi-Task ModernBERT...


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: answerdotai/ModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
head.dense.weight | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 
decoder.bias      | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



MODELLO MULTI-TASK PRONTO PER IL TRAINING
Architettura: ModernBERT-base (Custom ACOS Extractor)
Task: Token Classification + 2x Binary Classification (Impliciti)
Numero di Classi Token: 5
Optimizer: AdamW 8-bit (lr=5e-5)
Loss Function: Loss Combinata (calcolata internamente)

MODELLO PRONTO PER IL TRAINING
Architettura: ModernBERT-base
Task: Token Classification (Estrazione Aspetti & Opinioni)
Numero di Classi: 5
Optimizer: AdamW 8-bit (lr=5e-5)
Loss Function: CrossEntropyLoss


In [ ]:
# --- 1. CONFIGURAZIONE AVANZATA MEMORIA E WANDB ---
print("Attivazione Ottimizzazioni di Memoria...")

# Gradient Checkpointing: si applica all'encoder interno (BERT) della nostra classe custom
model_step1.bert.gradient_checkpointing_enable()

accumulation_steps = wandb.config.get('accumulation_steps', 4) 
patience = wandb.config.get('patience', 5)
epochs = wandb.config.get('epochs', 40)
lr = wandb.config.get('learning_rate', 2e-5)
patience_counter = 0

# Optimizer a 8-bit
optimizer = bnb.optim.AdamW8bit(model_step1.parameters(), lr=lr)

# Scaler per Mixed Precision (FP16)
scaler = GradScaler() 

total_steps = (len(train_loader_rest) // accumulation_steps) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=total_steps
)

# --- 2. FUNZIONI DI SUPPORTO OTTIMIZZATE (MULTI-TASK) ---

def evaluate_model_multitask(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            imp_asp_labels = batch['implicit_aspect_labels'].to(device)
            imp_opi_labels = batch['implicit_opinion_labels'].to(device)
            
            # Usiamo autocast in valutazione per risparmiare memoria
            with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(
                    input_ids=input_ids, 
                    attention_mask=attention_mask, 
                    labels=labels,
                    implicit_aspect_labels=imp_asp_labels,
                    implicit_opinion_labels=imp_opi_labels
                )
            
            total_loss += outputs['loss'].item()
    return total_loss / len(data_loader)

def train_epoch_multitask(model, data_loader, optimizer, scheduler, device, epoch_idx, scaler, accumulation_steps):
    model.train()
    total_loss = 0
    optimizer.zero_grad() # Reset iniziale
    
    loop = tqdm(data_loader, leave=True)
    
    for i, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        imp_asp_labels = batch['implicit_aspect_labels'].to(device)
        imp_opi_labels = batch['implicit_opinion_labels'].to(device)
        
        # A. Mixed Precision Forward Pass (FP16)
        with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
            outputs = model(
                input_ids=input_ids, 
                attention_mask=attention_mask, 
                labels=labels,
                implicit_aspect_labels=imp_asp_labels,
                implicit_opinion_labels=imp_opi_labels
            )
            # Normalizziamo la loss per l'accumulo dei gradienti
            loss = outputs['loss'] / accumulation_steps 
        
        # B. Backward Pass con Scaler (Evita underflow/overflow dell'FP16)
        scaler.scale(loss).backward()
        
        # C. Update Pesi ogni 'accumulation_steps'
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(data_loader):
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item() * accumulation_steps
        wandb.log({"batch_loss": loss.item() * accumulation_steps})
        loop.set_description(f"Epoca {epoch_idx + 1}")
        loop.set_postfix(loss=loss.item() * accumulation_steps)

    return total_loss / len(data_loader)

# --- 3. CICLO DI ADDESTRAMENTO ---

print(f"\nTraining Multi-Task su RESTAURANT: {epochs} epoche | Device: {device}")
print(f"Accumulo Gradienti: ogni {accumulation_steps} step | FP16: Attivato")

best_valid_loss_rest = float('inf')
output_dir = "./best_multitask_extractor_restaurant"

for epoch in range(epochs):
    print(f"\n--- Epoca {epoch+1}/{epochs} ---")
    
    # 1. Training
    train_loss_rest = train_epoch_multitask(model_step1, train_loader_rest, optimizer, scheduler, device, epoch, scaler, accumulation_steps)
    
    # 2. Validazione
    valid_loss_rest = evaluate_model_multitask(model_step1, dev_loader_rest, device)
    
    # Pulizia spietata della cache della GPU a fine epoca!
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print(f"Train Loss: {train_loss_rest:.4f} | Valid Loss: {valid_loss_rest:.4f}")
    
    # 3. Log metriche epoca su W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss_epoch": train_loss_rest,
        "valid_loss_epoch": valid_loss_rest
    })
    
    # --- LOGICA EARLY STOPPING & CHECKPOINT ---
    
    if valid_loss_rest < best_valid_loss_rest:
        best_valid_loss_rest = valid_loss_rest
        patience_counter = 0  
        
        print(f"Miglior modello trovato (Loss: {best_valid_loss_rest:.4f})")
        
        os.makedirs(output_dir, exist_ok=True)
        # Salvataggio Custom per l'architettura Multi-Task
        torch.save(model_step1.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))
        
    else:
        patience_counter += 1  
        print(f"Nessun miglioramento. Patience: {patience_counter}/{patience}")
        
        if patience_counter >= patience:
            print(f"\nEARLY STOPPING ATTIVATO! Interruzione all'epoca {epoch+1}.")
            break 

print("\nFine Addestramento Multi-Task.")
wandb.finish()

Attivazione Ottimizzazioni di Memoria...

Training Multi-Task su RESTAURANT: 40 epoche | Device: cuda
Accumulo Gradienti: ogni 4 step | FP16: Attivato

--- Epoca 1/40 ---


Epoca 1:   3%|▎         | 3/96 [00:00<00:24,  3.73it/s, loss=3.24]/home/al3th3ia/Scrivania/Cristina/venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Epoca 1: 100%|██████████| 96/96 [00:19<00:00,  4.82it/s, loss=1.91]


Train Loss: 2.2520 | Valid Loss: 1.7552
Miglior modello trovato (Loss: 1.7552)

--- Epoca 2/40 ---


Epoca 2: 100%|██████████| 96/96 [00:19<00:00,  5.03it/s, loss=0.983]


Train Loss: 1.4441 | Valid Loss: 1.2890
Miglior modello trovato (Loss: 1.2890)

--- Epoca 3/40 ---


...

Epoca 4: 100%|██████████| 96/96 [00:19<00:00,  4.99it/s, loss=0.771]


Train Loss: 0.5663 | Valid Loss: 1.1235
Nessun miglioramento. Patience: 1/2

--- Epoca 5/40 ---


Epoca 5: 100%|██████████| 96/96 [00:19<00:00,  4.99it/s, loss=0.232]


Train Loss: 0.3204 | Valid Loss: 1.1719
Nessun miglioramento. Patience: 2/2

EARLY STOPPING ATTIVATO! Interruzione all'epoca 5.

Fine Addestramento Multi-Task.


batch_loss,█▆▅▄▅▅▄▄▅▄▄▄▄▄▃▂▃▃▃▃▃▃▃▄▃▂▂▂▂▃▂▁▁▁▁▁▁▁▁▁
epoch,▁▃▅▆█
train_loss_epoch,█▅▃▂▁
valid_loss_epoch,█▃▁▁▂
batch_loss,0.23184
epoch,5
train_loss_epoch,0.32037
valid_loss_epoch,1.17191


In [ ]:
# --- A. CARICAMENTO DEL "CAMPIONE" MULTI-TASK ---
print("Caricamento del modello Multi-Task migliore...")

# 1. Inizializziamo la nostra architettura custom
model_step1 = ModernBertACOS_Extractor(num_labels=5)

# 2. Carichiamo i pesi salvati del miglior modello (solo i pesi, non l'intero oggetto)
model_path = "./best_multitask_extractor_restaurant/pytorch_model.bin"
model_step1.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))

model_step1.to(device)
model_step1.eval() # Modalità esame (spegne dropout)

# --- B. PREPARAZIONE METRICHE ---
# Carichiamo la metrica seqeval (standard per NER/ABSA)
metric = load("seqeval")

# Mappa per decodificare i numeri in etichette
id2label = {0: 'O', 1: 'B-ASP', 2: 'I-ASP', 3: 'B-OPI', 4: 'I-OPI'}

print("\nInizio Test Multi-Task sul Dataset Restaurant...")

# --- C. CICLO DI PREVISIONE ---
predictions_tokens = []
true_labels_tokens = []

# Liste per salvare le predizioni binarie (Impliciti)
true_imp_asp, pred_imp_asp = [], []
true_imp_opi, pred_imp_opi = [], []

with torch.no_grad():
    for batch in tqdm(test_loader_rest, desc="Test RESTAURANT"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        imp_asp_labels = batch['implicit_aspect_labels'].to(device)
        imp_opi_labels = batch['implicit_opinion_labels'].to(device)

        # 1. Il modello fa le sue 3 predizioni contemporaneamente
        with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
            outputs = model_step1(input_ids, attention_mask=attention_mask)
        
        # 2. Estraiamo i risultati dalle 3 testoline
        token_preds = torch.argmax(outputs['token_logits'], dim=-1)
        asp_preds = torch.argmax(outputs['imp_asp_logits'], dim=-1)
        opi_preds = torch.argmax(outputs['imp_opi_logits'], dim=-1)
        
        # Salviamo i risultati binari
        true_imp_asp.extend(imp_asp_labels.cpu().tolist())
        pred_imp_asp.extend(asp_preds.cpu().tolist())
        
        true_imp_opi.extend(imp_opi_labels.cpu().tolist())
        pred_imp_opi.extend(opi_preds.cpu().tolist())

        # 3. Convertiamo i numeri in etichette BIO (pulendo il padding)
        for i in range(len(labels)):
            true_label_row = []
            pred_label_row = []
            
            for j in range(len(labels[i])):
                # Ignoriamo i token di padding (dove attention_mask è 0 o la label è -100)
                if labels[i][j] != -100 and attention_mask[i][j] == 1: 
                    true_label_row.append(id2label[labels[i][j].item()])
                    pred_label_row.append(id2label[token_preds[i][j].item()])
            
            true_labels_tokens.append(true_label_row)
            predictions_tokens.append(pred_label_row)

# --- D. CALCOLO E STAMPA RISULTATI ---
results_seq = metric.compute(predictions=predictions_tokens, references=true_labels_tokens)

print("\n" + "="*60)
print("RISULTATI FINALI: TOKEN CLASSIFICATION (Parole Esplicite)")
print("="*60)

print(f"Overall Precision: {results_seq['overall_precision']:.4f}")
print(f"Overall Recall:    {results_seq['overall_recall']:.4f}")
print(f"Overall F1-Score:  {results_seq['overall_f1']:.4f}")

print("\nDettaglio per Classe (Quello che conta per il paper):")
print("-" * 50)
for key in results_seq.keys():
    if key in ['ASP', 'OPI']: 
        print(f"   {key}:")
        print(f"   Precision: {results_seq[key]['precision']:.4f}")
        print(f"   Recall:    {results_seq[key]['recall']:.4f}")
        print(f"   F1-Score:  {results_seq[key]['f1']:.4f}")
        print(f"   Support:   {results_seq[key]['number']}") 

print("\n" + "="*60)
print("RISULTATI FINALI: IDENTIFICAZIONE IMPLICITI (NULL)")
print("="*60)

# Metriche per Aspetti Impliciti
acc_asp = accuracy_score(true_imp_asp, pred_imp_asp)
print(f"Accuratezza Aspetti Impliciti: {acc_asp:.4f}")
print(classification_report(true_imp_asp, pred_imp_asp, target_names=["Esplicito (0)", "Implicito (1)"], zero_division=0))

print("-" * 50)

# Metriche per Opinioni Implicite
acc_opi = accuracy_score(true_imp_opi, pred_imp_opi)
print(f"Accuratezza Opinioni Implicite: {acc_opi:.4f}")
print(classification_report(true_imp_opi, pred_imp_opi, target_names=["Esplicito (0)", "Implicito (1)"], zero_division=0))
print("="*60)

Caricamento del modello Multi-Task migliore...


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: answerdotai/ModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
head.norm.weight  | UNEXPECTED |  | 
head.dense.weight | UNEXPECTED |  | 
decoder.bias      | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Inizio Test Multi-Task sul Dataset Restaurant...


Test RESTAURANT: 100%|██████████| 37/37 [00:04<00:00,  8.47it/s]



RISULTATI FINALI: TOKEN CLASSIFICATION (Parole Esplicite)
Overall Precision: 0.5287
Overall Recall:    0.6537
Overall F1-Score:  0.5845

Dettaglio per Classe (Quello che conta per il paper):
--------------------------------------------------
   ASP:
   Precision: 0.4705
   Recall:    0.6036
   F1-Score:  0.5288
   Support:   608
   OPI:
   Precision: 0.5873
   Recall:    0.7006
   F1-Score:  0.6390
   Support:   648

RISULTATI FINALI: IDENTIFICAZIONE IMPLICITI (NULL)
🔹 Accuratezza Aspetti Impliciti: 0.8285
               precision    recall  f1-score   support

Esplicito (0)       0.84      0.92      0.88       386
Implicito (1)       0.81      0.65      0.72       197

     accuracy                           0.83       583
    macro avg       0.82      0.78      0.80       583
 weighted avg       0.83      0.83      0.82       583

--------------------------------------------------
🔹 Accuratezza Opinioni Implicite: 0.8045
               precision    recall  f1-score   support

Esplic

## Classificatore Category-Sentiment (Extract-Classify-ACOS)

Implementiamo il **secondo stadio** dell'architettura proposta nel paper. Dopo aver estratto gli Aspetti e le Opinioni nello Step 1, ora dobbiamo capire a quale Categoria appartengono e qual è il loro Sentiment.

Il codice di preparazione è diviso in tre componenti fondamentali:

 1. Il Dataset PyTorch (`ACOSPairDataset`)

 2. L'Architettura Custom (`ModernBertACOSClassifier`)

 3. Inizializzazione e DataLoaders

### 1. Il Dataset PyTorch (`ACOSPairDataset`)
Questa classe si occupa di "impacchettare" i dati per la rete neurale. 
Legge le coppie salvate nei file `.pkl`, passa il testo nel Tokenizer di ModernBERT e trasforma tutto in **Tensori PyTorch**. Nota chiave: legge direttamente gli indici degli span (che ho già corretto con l'offset `+1` per far spazio al token `[CLS]`) e carica l'array di 121 etichette (le "soluzioni").

In [5]:
class ACOSPairDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row['review_text']
        
        # Gli span sono già corretti con il +1 per il [CLS]!
        a_span = row['aspect_span']
        o_span = row['opinion_span']

        # Tokenizzazione (ModernBERT usa il token [CLS] in automatico)
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        labels = torch.tensor(row['labels'], dtype=torch.long)

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'aspect_span': torch.tensor(a_span),
            'opinion_span': torch.tensor(o_span),
            'labels': labels
        }

### 2. L'Architettura Custom (`ModernBertACOSClassifier`)
Questa è la vera "magia" matematica del paper, tradotta in codice:
* **Il Corpo (Backbone):** Invece di partire da zero, carichiamo il *corpo* del modello che hai già addestrato nello Step 1 (`best_model_laptop`). In questo modo, la rete conosce già il dominio tecnico dei computer!
* **Span Pooling:** Il modello estrae i vettori (hidden states) corrispondenti alle parole dell'Aspetto e dell'Opinione e ne calcola la media. Se un elemento è implicito (`-1`), pesca automaticamente il vettore globale del token `[CLS]`.
* **Feature Fusion:** Concatena il vettore dell'aspetto ($u_a$) e dell'opinione ($u_o$) in un unico grande vettore di dimensione 1536.
* **Le 121 Teste (Multiple Multi-class):** Passa questo vettore in 121 classificatori lineari paralleli. Ognuno di essi deciderà se per la *sua* categoria la coppia è `Positive (0)`, `Negative (1)`, `Neutral (2)` o `Invalid (3)`.

In [6]:
class ModernBertACOSClassifier(nn.Module):
    def __init__(self, path_to_best_model, num_categories):
        super(ModernBertACOSClassifier, self).__init__()
        
        # Carichiamo SOLO IL CORPO dal tuo modello dello Step 1
        self.modernbert = AutoModel.from_pretrained(path_to_best_model)
        hidden_size = self.modernbert.config.hidden_size # 768
        
        # Le 121 teste (ognuna prende il vettore concatenato 1536 e sputa 4 classi)
        self.heads = nn.ModuleList([
            nn.Linear(hidden_size * 2, 4) for _ in range(num_categories)
        ])
        
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask, aspect_spans, opinion_spans):
        outputs = self.modernbert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state # [Batch, Seq_Len, 768]
        
        batch_size = last_hidden_state.size(0)
        u_a_list, u_o_list = [], []

        for i in range(batch_size):
            # Pooling Aspetto
            a_start, a_end = aspect_spans[i]
            if a_start == -1: 
                u_a = last_hidden_state[i, 0, :] # Token [CLS]
            else:
                u_a = last_hidden_state[i, a_start:a_end, :].mean(dim=0)
            
            # Pooling Opinione
            o_start, o_end = opinion_spans[i]
            if o_start == -1: 
                u_o = last_hidden_state[i, 0, :] # Token [CLS]
            else:
                u_o = last_hidden_state[i, o_start:o_end, :].mean(dim=0)
            
            u_a_list.append(u_a)
            u_o_list.append(u_o)

        # Concatenazione: [u_a ; u_o]
        combined_features = torch.cat((torch.stack(u_a_list), torch.stack(u_o_list)), dim=-1)
        combined_features = self.dropout(combined_features)

        # Passiamo il vettore nelle teste lineari
        logits = [head(combined_features) for head in self.heads]
        
        # Output: [Batch, 121, 4]
        return torch.stack(logits, dim=1)

### 3. Inizializzazione e DataLoaders
L'ultimo blocco carica fisicamente i file salvati dalla nostra "Fabbrica dei Dati", istanzia i `Dataset`, e crea i `DataLoader` (con batch size = 16) per "nutrire" la GPU in modo efficiente durante l'addestramento. Infine, sposta il modello sulla scheda video (CUDA) pronto per il training.

In [7]:
# 1. Carichiamo la lista delle categorie salvata prima
with open("data_coppie/restaurant_categories.pkl", "rb") as f:
    category_list = pickle.load(f)
num_categories = len(category_list) # 121

# 2. Carichiamo i DataFrame di Train e Dev
df_train = pd.read_pickle("data_coppie/train_restaurant_pairs.pkl")
df_dev = pd.read_pickle("data_coppie/dev_restaurant_pairs.pkl")
df_test = pd.read_pickle("data_coppie/test_restaurant_pairs.pkl")

# 3. Inizializziamo il Tokenizer e i Dataset
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")
train_dataset = ACOSPairDataset(df_train, tokenizer)
dev_dataset = ACOSPairDataset(df_dev, tokenizer)
test_dataset = ACOSPairDataset(df_test, tokenizer) 

# 4. Creiamo i DataLoader (Batch size 16 è un buon compromesso tra velocità e VRAM)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(dev_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16) 
# 5. Inizializziamo il Modello usando i pesi dello Step 1!
model = ModernBertACOSClassifier("./best_model_restaurant", num_categories)
model.to(device)

print(f"Modello inizializzato! Categorie: {num_categories} | Device: {device}")

Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: ./best_model_restaurant
Key               | Status     |  | 
------------------+------------+--+-
classifier.weight | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 
head.dense.weight | UNEXPECTED |  | 
classifier.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Modello inizializzato! Categorie: 13 | Device: mps


## WANDB per lo step 2

In [6]:
# Chiudiamo per sicurezza qualsiasi run precedente rimasta aperta nello stesso notebook
if wandb.run is not None:
    wandb.finish()

WANDB_ENTITY = "cristinatextmining"

# 1. Definizione degli Hyperparameters per lo STEP 2
config_step2 = {
    "learning_rate": 2e-5, # Solitamente per lo Step 2 un LR leggermente più basso è meglio (es. 2e-5)
    "epochs": 40,
    "batch_size": 16,
    "accumulation_steps": 4, # Aggiunto per il tuo training loop ottimizzato!
    "model_name": "answerdotai/ModernBERT-base",
    "dataset": "Restaurant-ACOS", 
    "seed": 42,
    "patience": 5  # Per Early Stopping
}

# 2. Inizializzazione del Run per lo Step 2
wandb.init(
    project="BigData-TextMining-ACOS",
    entity=WANDB_ENTITY,
    config=config_step2,
    # Aggiungiamo "Step2_Class" al nome per distinguerlo dallo Step 1
    name=f"Step2_Class_{config_step2['model_name']}_{config_step2['dataset']}" 
)

print(f" W&B inizializzato per il progetto: {wandb.run.project}")
print(f"Nome della Run attuale: {wandb.run.name}")

wandb: Currently logged in as: cristinatomaciello2001 (cristinatextmining) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


 W&B inizializzato per il progetto: BigData-TextMining-ACOS
Nome della Run attuale: Step2_Class_answerdotai/ModernBERT-base_Restaurant-ACOS


## Train su Sentiment e Categoria

In [7]:
# --- 1. CONFIGURAZIONE AVANZATA MEMORIA ---
# Gradient Checkpointing: abilitato SOLO sul corpo di ModernBERT
model.modernbert.gradient_checkpointing_enable()

# Parametri per simulare un batch size maggiore
accumulation_steps = config_step2.get('accumulation_steps', 4) 
patience = config_step2.get('patience', 5)
patience_counter = 0

# Ottimizzatore AdamW a 8-bit
optimizer = bnb.optim.AdamW8bit(
    model.parameters(), 
    lr=config_step2['learning_rate']
)

# Scaler per Mixed Precision (fondamentale per evitare l'OOM)
scaler = GradScaler() 

# Un bilanciamento più "umano": i sentimenti contano 10 volte più della classe Invalido.
# Questo costringe il modello a essere molto più cauto.
weights = torch.tensor([10.0, 10.0, 10.0, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

# Calcolo degli step totali per lo scheduler
total_steps = (len(train_loader) // accumulation_steps) * config_step2['epochs']
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=total_steps
)

# --- 2. FUNZIONI DI SUPPORTO OTTIMIZZATE (STEP 2) ---

def evaluate_model_step2(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            aspect_span = batch['aspect_span'].to(device)
            opinion_span = batch['opinion_span'].to(device)
            labels = batch['labels'].to(device)
            
            with autocast(device_type='cuda'):
                logits = model(input_ids=input_ids, attention_mask=attention_mask, 
                               aspect_spans=aspect_span, opinion_spans=opinion_span)
                
                loss = criterion(logits.view(-1, 4), labels.view(-1))
            
            total_loss += loss.item()
            
    return total_loss / len(data_loader)

def train_epoch_step2(model, data_loader, optimizer, scheduler, criterion, device, epoch_idx, scaler, accumulation_steps):
    model.train()
    total_loss = 0
    optimizer.zero_grad() 
    
    loop = tqdm(data_loader, leave=True)
    
    for i, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        aspect_span = batch['aspect_span'].to(device)
        opinion_span = batch['opinion_span'].to(device)
        labels = batch['labels'].to(device)
        
        with autocast(device_type='cuda'):
            logits = model(input_ids=input_ids, attention_mask=attention_mask, 
                           aspect_spans=aspect_span, opinion_spans=opinion_span)
            
            loss = criterion(logits.view(-1, 4), labels.view(-1))
            loss = loss / accumulation_steps 
        
        scaler.scale(loss).backward()
        
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(data_loader):
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()
        
        real_loss = loss.item() * accumulation_steps
        total_loss += real_loss
        
        wandb.log({"batch_loss": real_loss})
        loop.set_description(f"Epoca {epoch_idx + 1}")
        loop.set_postfix(loss=real_loss)

    return total_loss / len(data_loader)

# --- 3. CICLO DI ADDESTRAMENTO ---

print(f"Training STEP 2 su RESTAURANT: {config_step2['epochs']} epoche | Device: {device}")
print(f"Accumulo Gradienti ogni {accumulation_steps} step | FP16 Attivato")

best_valid_loss_restaurant = float('inf')

for epoch in range(config_step2['epochs']):
    print(f"\n--- Epoca {epoch+1}/{config_step2['epochs']} ---")
    
    # 1. Training
    train_loss_restaurant = train_epoch_step2(
        model, train_loader, optimizer, scheduler, criterion, 
        device, epoch, scaler, accumulation_steps
    )
    
    # 2. Validazione
    valid_loss_restaurant = evaluate_model_step2(model, val_loader, criterion, device)
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print(f"Train Loss: {train_loss_restaurant:.4f} | Valid Loss: {valid_loss_restaurant:.4f}")
    
    # 3. Log metriche epoca su W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss_epoch": train_loss_restaurant,
        "valid_loss_epoch": valid_loss_restaurant
    })
    
    # --- LOGICA EARLY STOPPING & CHECKPOINT ---
    
    if valid_loss_restaurant < best_valid_loss_restaurant:
        best_valid_loss_restaurant = valid_loss_restaurant
        patience_counter = 0  
        
        print(f"Miglior modello trovato (Loss: {best_valid_loss_restaurant:.4f})! Salvataggio...")
        
        # Salvataggio custom model nella cartella dei RESTAURANT
        save_dir = "./best_classifier_restaurant"
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            
        torch.save(model.state_dict(), os.path.join(save_dir, "pytorch_model.bin"))
        
    else:
        patience_counter += 1  
        print(f"Nessun miglioramento. Patience: {patience_counter}/{patience}")
        
        if patience_counter >= patience:
            print(f"\nEARLY STOPPING ATTIVATO! Interruzione all'epoca {epoch+1}.")
            break 

print("\nFine Addestramento Step 2 (Restaurant).")
wandb.finish()

Training STEP 2 su RESTAURANT: 40 epoche | Device: cuda
Accumulo Gradienti ogni 4 step | FP16 Attivato

--- Epoca 1/40 ---


Epoca 1: 100%|██████████| 232/232 [00:49<00:00,  4.73it/s, loss=0.396]


Train Loss: 0.8031 | Valid Loss: 0.6246
Miglior modello trovato (Loss: 0.6246)! Salvataggio...

--- Epoca 2/40 ---


Epoca 2: 100%|██████████| 232/232 [00:49<00:00,  4.71it/s, loss=1.18] 


Train Loss: 0.4954 | Valid Loss: 0.4818
Miglior modello trovato (Loss: 0.4818)! Salvataggio...

--- Epoca 3/40 ---


...

Epoca 10: 100%|██████████| 232/232 [00:50<00:00,  4.62it/s, loss=0.018] 


Train Loss: 0.0597 | Valid Loss: 0.4369
Nessun miglioramento. Patience: 4/5

--- Epoca 11/40 ---


Epoca 11: 100%|██████████| 232/232 [00:49<00:00,  4.65it/s, loss=0.0986]


Train Loss: 0.0550 | Valid Loss: 0.4442
Nessun miglioramento. Patience: 5/5

EARLY STOPPING ATTIVATO! Interruzione all'epoca 11.

Fine Addestramento Step 2 (Restaurant).


batch_loss,█▇▆▆▄▃▄▃▄▃▂▂▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▂▂▃▄▅▅▆▇▇█
train_loss_epoch,█▅▄▃▂▂▁▁▁▁▁
valid_loss_epoch,█▄▁▁▁▁▁▂▃▂▃
batch_loss,0.09864
epoch,11
train_loss_epoch,0.05499
valid_loss_epoch,0.44417


## Test su Sentiment e Categoria

In [8]:
# Chiudiamo run appese
if wandb.run is not None:
    wandb.finish()

WANDB_ENTITY = "cristinatextmining"

wandb.init(
    project="BigData-TextMining-ACOS",
    entity=WANDB_ENTITY,
    name=f"TEST_Step2_{config_step2['model_name']}_{config_step2['dataset']}",
    job_type="test"
)

# Ricreiamo l'architettura del modello
num_categories = len(category_list)
model_test = ModernBertACOSClassifier("./best_model_restaurant", num_categories)

# Carichiamo i pesi dello Step 2
model_path = "./best_classifier_restaurant/pytorch_model.bin"
state_dict = torch.load(model_path, map_location=device)

# USIAMO STRICT=FALSE
missing_keys, unexpected_keys = model_test.load_state_dict(state_dict, strict=False)

print("\n--- CHECK CARICAMENTO PESI ---")
print(f"Chiavi Inaspettate (OK se sono dello Step 1): {len(unexpected_keys)}")
print(f"Chiavi Mancanti (PROBLEMA se sono 'heads'): {len(missing_keys)}")

heads_missing = [k for k in missing_keys if "heads" in k]
if heads_missing:
    print(f"ERRORE CRITICO: Le teste di classificazione non sono state caricate! {heads_missing[:5]}")
else:
    print("SUCCESS: Le teste di classificazione (Sentiment) sono state caricate correttamente.")
    
model_test.to(device)
model_test.eval()

# --- 1. ESTRAZIONE PROBABILITA' ---
print("⚙️ Estrazione di tutte le probabilità dal modello in corso (attendere)...")

all_probs_list = []
all_true_list = []

with torch.no_grad():
    for batch in test_loader: 
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        aspect_span = batch['aspect_span'].to(device)
        opinion_span = batch['opinion_span'].to(device)
        labels = batch['labels'].to(device) 
        
        # Gestione sicura dell'autocast basata sull'hardware
        if device.type == 'cuda':
            with torch.amp.autocast(device_type='cuda'):
                logits = model_test(input_ids, attention_mask, aspect_span, opinion_span)
        else:
            # Per Mac MPS o CPU facciamo il forward pass standard
            logits = model_test(input_ids, attention_mask, aspect_span, opinion_span)
            
        probs = torch.softmax(logits, dim=-1).cpu() 
        
        all_probs_list.append(probs)
        all_true_list.append(labels.cpu())

all_probs = torch.cat(all_probs_list, dim=0) 
all_true = torch.cat(all_true_list, dim=0).numpy().flatten()


# --- 2. GRID SEARCH SUL THRESHOLD (OTTIMIZZATA PER MACRO F1) ---
print("\n🔍 Avvio Grid Search per la migliore Soglia di Confidenza (Macro F1)...")

thresholds_to_test = np.arange(0.50, 1.00, 0.01) 
best_micro_f1 = 0.0
best_macro_f1 = 0.0
best_threshold = 0.0
best_report = ""

target_names = ['Positive (0)', 'Negative (1)', 'Neutral (2)']
labels_to_eval = [0, 1, 2]

for thresh in thresholds_to_test:
    valid_class_probs, valid_class_preds = torch.max(all_probs[:, :, :3], dim=-1)
    final_preds = torch.full_like(valid_class_preds, 3)
    
    mask = valid_class_probs > thresh
    final_preds[mask] = valid_class_preds[mask]
    
    preds_flat = final_preds.numpy().flatten()
    
    current_micro_f1 = f1_score(all_true, preds_flat, labels=labels_to_eval, average='micro')
    current_macro_f1 = f1_score(all_true, preds_flat, labels=labels_to_eval, average='macro')
    
    if current_macro_f1 > best_macro_f1:
        best_macro_f1 = current_macro_f1
        best_micro_f1 = current_micro_f1 
        best_threshold = thresh
        best_report = classification_report(
            all_true, preds_flat, labels=labels_to_eval, 
            target_names=target_names, zero_division=0
        )


# --- 3. STAMPA DEI RISULTATI VINCITORI E LOG W&B ---
print(f"IL MILGIOR THRESHOLD E'= {best_threshold:.2f} ({(best_threshold*100):.0f}%)")

print("\n--- MIGLIOR CLASSIFICATION REPORT ---")
print(best_report)
print(f"BEST MICRO F1-Score: {best_micro_f1:.4f}")
print(f"CORRISPONDENTE MACRO F1: {best_macro_f1:.4f}")
print("="*50)

wandb.log({
    "best_test_micro_f1": best_micro_f1,
    "best_test_macro_f1": best_macro_f1,
    "optimal_threshold": best_threshold
})
wandb.finish()

Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: ./best_model_restaurant
Key               | Status     |  | 
------------------+------------+--+-
head.dense.weight | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 
classifier.bias   | UNEXPECTED |  | 
classifier.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



--- CHECK CARICAMENTO PESI ---
Chiavi Inaspettate (OK se sono dello Step 1): 0
Chiavi Mancanti (PROBLEMA se sono 'heads'): 0
SUCCESS: Le teste di classificazione (Sentiment) sono state caricate correttamente.
⚙️ Estrazione di tutte le probabilità dal modello in corso (attendere)...

🔍 Avvio Grid Search per la migliore Soglia di Confidenza (Macro F1)...
IL MILGIOR THRESHOLD E'= 0.53 (53%)

--- MIGLIOR CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

Positive (0)       0.46      0.57      0.51       205
Negative (1)       1.00      0.02      0.04        44
 Neutral (2)       0.59      0.69      0.64       667

   micro avg       0.56      0.63      0.59       916
   macro avg       0.68      0.43      0.40       916
weighted avg       0.58      0.63      0.58       916

BEST MICRO F1-Score: 0.5915
CORRISPONDENTE MACRO F1: 0.3958


best_test_macro_f1,▁
best_test_micro_f1,▁
optimal_threshold,▁
best_test_macro_f1,0.39578
best_test_micro_f1,0.59149
optimal_threshold,0.53


### Test sulla quadruplice completa (Aspetto-Opinione-Categoria-Sentiment)
In questa cella, eseguiamo il test finale del nostro modello su tutte e quattro le dimensioni del problema: Aspetto, Opinione, Categoria e Sentiment.
Il processo è il seguente:
1. **Iterazione sui Dati di Test:** Per ogni batch di coppie (Aspetto-Opinione) estratte dallo Step 1, il modello predice la Categoria e il Sentiment.
2. **Raccolta delle Predizioni:** Le predizioni vengono raccole in liste separate per le categorie e i sentimenti, insieme alle etichette vere.
3. **Calcolo delle Metriche:** Utilizziamo `accuracy_score` e `classification_report` di `sklearn` per valutare le prestazioni del modello sia sulla Categoria che sul Sentiment, considerando solo le coppie che sono state correttamente estratte nello Step 1 (ovvero quelle con etichetta diversa da "Invalid (3)").
4. **Stampa dei Risultati:** I risultati vengono stampati in modo chiaro, mostrando l'accuratezza e il report dettagliato per entrambe le dimensioni.

In [8]:
# ==========================================
# 1. CARICAMENTO DEI MODELLI E PREPARATIVI
# ==========================================
print("Caricamento configurazioni...")
with open("data_coppie/restaurant_categories.pkl", "rb") as f:
    category_list = pickle.load(f)
num_categories = len(category_list)
id2label = {0: 'O', 1: 'B-ASP', 2: 'I-ASP', 3: 'B-OPI', 4: 'I-OPI'}

print("Caricamento Step 1 (L'Investigatore Multi-Task)...")
model_step1 = ModernBertACOS_Extractor(num_labels=5).to(device)
model_step1.load_state_dict(torch.load("./best_multitask_extractor_restaurant/pytorch_model.bin", map_location=device, weights_only=True))
model_step1.eval()

print("Caricamento Step 2 (Lo Psicologo Classificatore)...")
model_step2 = ModernBertACOSClassifier("./best_model_restaurant", num_categories).to(device)
model_step2.load_state_dict(torch.load("./best_classifier_restaurant/pytorch_model.bin", map_location=device, weights_only=True))
model_step2.eval()

# ==========================================
# 2. FUNZIONI DI SUPPORTO PER L'ESTRAZIONE
# ==========================================
def get_spans(tags, b_tag, i_tag):
    spans = []
    start = -1
    for i, tag in enumerate(tags):
        if tag == b_tag:
            if start != -1: spans.append((start, i))
            start = i
        elif tag == i_tag and start != -1: continue
        else:
            if start != -1:
                spans.append((start, i))
                start = -1
    if start != -1: spans.append((start, len(tags)))
    return spans

def predict_quadruples_e2e(text, model_1, model_2, tokenizer, cat_list):
    """La vera Pipeline End-to-End su una singola frase."""
    words = text.split()
    inputs = tokenizer(words, is_split_into_words=True, return_tensors="pt", truncation=True, max_length=128, padding='max_length').to(device)
    
    # --- FASE 1: L'Investigatore ---
    with torch.no_grad():
        with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
            out1 = model_1(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        
    token_preds = torch.argmax(out1['token_logits'], dim=-1)[0].cpu().numpy()
    # Al posto di usare argmax, leggiamo le probabilità
    probs_imp_asp = torch.softmax(out1['imp_asp_logits'], dim=-1)[0]
    probs_imp_opi = torch.softmax(out1['imp_opi_logits'], dim=-1)[0]
    
    # Se il modello è sicuro almeno al 30% che c'è un implicito, fidiamoci!
    # Questo contrasta la sua "timidezza" dovuta all'overfitting
    imp_asp = 1 if probs_imp_asp[1].item() > 0.30 else 0
    imp_opi = 1 if probs_imp_opi[1].item() > 0.30 else 0
    
    word_ids = inputs.word_ids()
    word_tags = ["O"] * len(words)
    
    for idx, w_id in enumerate(word_ids):
        if w_id is not None and w_id < len(words) and word_tags[w_id] == "O":
            word_tags[w_id] = id2label[token_preds[idx]]
                
    asp_spans = get_spans(word_tags, "B-ASP", "I-ASP")
    opi_spans = get_spans(word_tags, "B-OPI", "I-OPI")
    
    # Se il modello segnala impliciti, aggiungiamo la coordinata speciale
    if imp_asp == 1 or len(asp_spans) == 0: asp_spans.append((-1, -1))
    if imp_opi == 1 or len(opi_spans) == 0: opi_spans.append((-1, -1))
    
    asp_spans = list(set(asp_spans))
    opi_spans = list(set(opi_spans))
    
    quadruples = []
    
    # --- FASE 2: Lo Psicologo ---
    for a in asp_spans:
        for o in opi_spans:
            fixed_a = (a[0]+1, a[1]+1) if a != (-1, -1) else (-1, -1)
            fixed_o = (o[0]+1, o[1]+1) if o != (-1, -1) else (-1, -1)
            
            t_a = torch.tensor([fixed_a]).to(device)
            t_o = torch.tensor([fixed_o]).to(device)
            
            with torch.no_grad():
                with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                    out2 = model_2(input_ids=inputs['input_ids'], 
                                   attention_mask=inputs['attention_mask'], 
                                   aspect_spans=t_a, 
                                   opinion_spans=t_o)
            
            logits = out2['logits'] if isinstance(out2, dict) else out2 
            probs = torch.softmax(logits[0], dim=-1) 
            
            for cat_idx, prob_dist in enumerate(probs):
                best_class = torch.argmax(prob_dist).item()
                
                # --- FILTRO DI SICUREZZA ---
                # 1. Se vince la classe 3 (Invalido), scartiamo
                if best_class == 3:
                    continue
                
                # 2. Soglia di confidenza: se il sentimento non ha almeno il 50%, scartiamo
                if prob_dist[best_class] < 0.5:
                    continue
                
                quadruples.append({
                    'aspect_span': a,
                    'opinion_span': o,
                    'category': cat_list[cat_idx],
                    'sentiment': best_class
                })
                    
    return quadruples

# ==========================================
# 3. TEST SULL'INTERO DATASET RESTAURANT
# ==========================================
print("\nAvvio Valutazione End-to-End sul Test Set...")
true_quads = []
pred_quads = []

percorso_file = os.path.join("data_parsing", "test_rest_parsed.pkl")
test_rest_parsed = pd.read_pickle(percorso_file)

for idx, row in tqdm(test_rest_parsed.iterrows(), total=len(test_rest_parsed), desc="Analisi Frasi"):
    text = row['review_text']
    preds = predict_quadruples_e2e(text, model_step1, model_step2, tokenizer, category_list)
    
    p_set = set()
    for q in preds:
        p_set.add((tuple(q['aspect_span']), q['category'], tuple(q['opinion_span']), q['sentiment']))
    pred_quads.append(p_set)
    
    t_set = set()
    for q in row['parsed_quadruples']:
        a = tuple(q.get('span_A', [-1, -1]))
        o = tuple(q.get('span_B', [-1, -1]))
        c = q['category_aspect']
        s = int(q['sentiment'])
        t_set.add((a, c, o, s))
    true_quads.append(t_set)

# Metriche finali
total_pred = sum(len(p) for p in pred_quads)
total_true = sum(len(t) for t in true_quads)
correct = sum(len(p_set.intersection(t_set)) for p_set, t_set in zip(pred_quads, true_quads))

precision = correct / total_pred if total_pred > 0 else 0
recall = correct / total_true if total_true > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print("\n" + "="*50)
print("RISULTATI FINALI EXACT MATCH ACOS (RESTAURANT)")
print("="*50)
print(f"Quadruple Reali totali:    {total_true}")
print(f"Quadruple Predette totali: {total_pred}")
print(f"Quadruple Esatte:          {correct}")
print("-" * 50)
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print("="*50)

# Confronto con il benchmark del paper
if f1 > 0.4461:
    print("INCREDIBILE! Hai superato il paper originale (0.4461)!")
else:
    print("Ottimo lavoro! Il modello è ora bilanciato.")

Caricamento configurazioni...
Caricamento Step 1 (L'Investigatore Multi-Task)...


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: answerdotai/ModernBERT-base
Key               | Status     |  | 
------------------+------------+--+-
head.dense.weight | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 
decoder.bias      | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Caricamento Step 2 (Lo Psicologo Classificatore)...


Loading weights:   0%|          | 0/134 [00:00<?, ?it/s]

ModernBertModel LOAD REPORT from: ./best_model_restaurant
Key               | Status     |  | 
------------------+------------+--+-
classifier.weight | UNEXPECTED |  | 
head.norm.weight  | UNEXPECTED |  | 
head.dense.weight | UNEXPECTED |  | 
classifier.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Avvio Valutazione End-to-End sul Test Set...


Analisi Frasi: 100%|██████████| 583/583 [01:38<00:00,  5.89it/s]


RISULTATI FINALI EXACT MATCH ACOS (RESTAURANT)
Quadruple Reali totali:    916
Quadruple Predette totali: 999
Quadruple Esatte:          252
--------------------------------------------------
Precision: 0.2523
Recall:    0.2751
F1-Score:  0.2632
Ottimo lavoro! Il modello è ora bilanciato.


In [10]:
# --- DEBUG: GUARDIE E LADRI SULLA PRIMA FRASE ---
test_row = test_rest_parsed.iloc[2] 
test_text = test_row['review_text']

print(f"📝 TESTO: {test_text}")
print("\n✅ QUADRUPLE VERE (Ground Truth):")
for q in test_row['parsed_quadruples']:
    print(f"   Aspetto: {q.get('span_A', [-1,-1])} | Opinione: {q.get('span_B', [-1,-1])} | Categoria: {q['category_aspect']} | Sent: {q['sentiment']}")

print("\n🤖 QUADRUPLE PREDETTE DAL MODELLO:")
preds = predict_quadruples_e2e(test_text, model_step1, model_step2, tokenizer, category_list)
for p in preds:
    print(f"   Aspetto: {p['aspect_span']} | Opinione: {p['opinion_span']} | Categoria: {p['category']} | Sent: {p['sentiment']}")

📝 TESTO: not the biggest portions but adequate .

✅ QUADRUPLE VERE (Ground Truth):
   Aspetto: (3, 4) | Opinione: (0, 3) | Categoria: FOOD#STYLE_OPTIONS | Sent: 1
   Aspetto: (3, 4) | Opinione: (5, 6) | Categoria: FOOD#STYLE_OPTIONS | Sent: 1

🤖 QUADRUPLE PREDETTE DAL MODELLO:
   Aspetto: (-1, -1) | Opinione: (5, 6) | Categoria: RESTAURANT#GENERAL | Sent: 2
   Aspetto: (-1, -1) | Opinione: (5, 6) | Categoria: SERVICE#GENERAL | Sent: 0
